# Session 4: 벡터와 차원, 그리고 평가 (Integrated)

## 🎯 학습 목표
- **Deep Theory**: 벡터의 내적과 코사인 유사도, 오차 행렬의 기하학적 의미를 배웁니다.
- **Practice**: `numpy`의 행렬 연산을 이용해 코사인 유사도를 직접 구현합니다.
- **Concepts**: RAG 검색 품질 평가 지표(Precision, Recall, MRR)를 익힙니다.
- **Experiment**: 차원의 저주(Curse of Dimensionality)를 시뮬레이션으로 체감합니다.

---

# Part 1. 📐 Deep Theory: 벡터 공간

## 1. 내적 (Dot Product)과 유사도

두 벡터 $A, B$의 내적은 다음과 같습니다.
$$ A \cdot B = \sum a_i b_i = |A||B|\cos\theta $$

여기서 $\cos\theta$를 남기고 정리하면 **코사인 유사도(Cosine Similarity)**가 됩니다.
$$ \text{Sim}(A, B) = \cos\theta = \frac{A \cdot B}{|A||B|} $$

> **Insight**: 내적은 두 벡터가 **'얼마나 같은 방향을 가리키는가'**를 나타냅니다. 크기(|A|)를 정규화하면 방향성만 비교할 수 있습니다.

## 2. 평가 지표 (Evaluation Metrics)

- **Precision (정밀도)**: 모델이 True라고 한 것 중 진짜 True의 비율. (거짓 양성을 피하고 싶을 때)
- **Recall (재현율)**: 실제 True인 것 중 모델이 찾은 것의 비율. (놓치지 않는 게 중요할 때)
- **F1-score**: Precision과 Recall의 **조화 평균(Harmonic Mean)**.
    $$ F1 = 2 \cdot \frac{P \cdot R}{P + R} $$
    산술 평균이 아니라 조화 평균을 쓰는 이유는, 둘 중 하나가 0에 가까우면 전체 점수도 0으로 만들기 위함입니다.

---
# Part 2. 💻 Manual Implementation: 코사인 유사도

`numpy.dot`, `numpy.linalg.norm`을 사용하여 코사인 유사도 함수를 만듭니다.

In [ ]:
import numpy as np

def cosine_similarity(v1, v2):
    # TODO: 코사인 유사도 수식 구현
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    
    if norm_v1 == 0 or norm_v2 == 0:
        return 0
        
    return dot_product / (norm_v1 * norm_v2)

# 테스트
vec_a = np.array([1, 2, 3])
vec_b = np.array([1, 2, 3]) # 동일 -> 1
vec_c = np.array([-1, -2, -3]) # 반대 -> -1

print(f"Sim(A, B): {cosine_similarity(vec_a, vec_b):.2f}")
print(f"Sim(A, C): {cosine_similarity(vec_a, vec_c):.2f}")

---
# 🚀 Practical Usage: Sklearn Metrics

코사인 유사도나 평가 지표를 직접 짤 일은 드뭅니다. 검증된 라이브러리를 사용하세요.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as sk_cosine_sim
from sklearn.metrics import f1_score, precision_score, recall_score

# 1. Cosine Similarity (행렬 단위 최적화)
# (n_samples, n_features) 형태여야 함
a = vec_a.reshape(1, -1)
b = vec_b.reshape(1, -1)
print(f"Sklearn Cosine: {sk_cosine_sim(a, b)[0][0]:.2f}")

# 2. Evaluation Metrics
y_true = [0, 1, 1, 0, 1, 1]
y_pred = [0, 1, 0, 0, 1, 1]

print(f"Precision: {precision_score(y_true, y_pred):.2f}")
print(f"Recall: {recall_score(y_true, y_pred):.2f}")
print(f"F1-Score: {f1_score(y_true, y_pred):.2f}")

---
# Part 3. 📖 Concepts & Practice: 행렬 연산과 RAG

RAG에서는 1개의 쿼리 벡터와 N개의 문서 벡터 간의 유사도를 한꺼번에 구해야 합니다.
이때 **행렬 곱셈(Matrix Multiplication)**을 사용하면 매우 빠릅니다.

$$ Q \cdot D^T $$

In [ ]:
# 쿼리 1개 (차원 128)
query = np.random.rand(1, 128)
# 문서 1000개 (차원 128)
documents = np.random.rand(1000, 128)

# 한 번에 계산 (Broadcasting)
# 코사인 유사도를 위해 정규화 먼저 수행
query_norm = query / np.linalg.norm(query, axis=1, keepdims=True)
doc_norm = documents / np.linalg.norm(documents, axis=1, keepdims=True)

# 내적 (1x128) @ (128x1000) -> (1x1000)
similarities = np.dot(query_norm, doc_norm.T)

print(f"Calculated {similarities.shape[1]} scores instantly.")
print(f"Top 5 scores: {np.sort(similarities[0])[-5:]}")

---
# Part 4. 🧪 Experiment: 차원의 저주 (Curse of Dimensionality)

### Scenario: 고차원이 되면 무슨 일이 벌어질까?
차원이 높아질수록 공간이 '희소(Sparse)'해져서, 무작위로 뽑은 두 점 사이의 거리가 점점 비슷해집니다. 
즉, **유사도 변별력이 사라집니다**.

In [ ]:
import matplotlib.pyplot as plt

dims = [2, 10, 100, 1000, 10000]
avg_distances = []

for d in dims:
    # 무작위 벡터 100개 생성
    vecs = np.random.rand(100, d)
    
    # 모든 쌍 사이의 거리 계산
    dists = []
    for i in range(100):
        for j in range(i+1, 100):
            dist = np.linalg.norm(vecs[i] - vecs[j])
            dists.append(dist)
            
    # 평균 거리 기록 (차원 크기에 따른 정규화 없이 단순 거리)
    avg_distances.append(np.mean(dists))

plt.plot(dims, avg_distances, marker='o')
plt.xscale('log')
plt.title("Average Distance between Random Points")
plt.xlabel("Dimension")
plt.ylabel("Avg Distance")
plt.show()

> **Discussion**: 차원이 커질수록 점들 사이의 거리가 계속 멀어집니다. 따라서 고차원 벡터 검색(Vector Search)에서는 차원 축소나 적절한 인덱싱이 중요합니다.